In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

%matplotlib inline

In [35]:
titanic_df = pd.read_csv("train_titanic.csv")
titanic_df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [37]:
df = titanic_df[['PassengerId', 'Pclass','Age', 'Survived']]
# df = titanic_df[['PassengerId', 'Pclass', 'Sex','Age', 'Survived']]
df = df.dropna()
dictSex = {'female': 1, 'male': 0}
# df['Sex'] = df['Sex'].apply(lambda x : 1 if x == 'female' else 0)
myX = df[['PassengerId', 'Pclass', 'Age']]
# myX = df[['PassengerId', 'Pclass', 'Sex','Age']]
survived = df['Survived']

xTrain, xTest, yTrain, yTest = train_test_split(myX, survived, test_size = 0.2)
train_df = pd.concat([xTrain, yTrain], axis=1)
test_df = pd.concat([xTest, yTest], axis=1)

In [6]:
model = LogisticRegression(fit_intercept=True) #generate a logistic regression model
model.fit(xTrain, yTrain) #fit the model
yPred = model.predict(xTest) 
score = accuracy_score(yTest, yPred)
print("The accuracy of the Logistic Regression model:", score)

The accuracy of the Logistic Regression model: 0.7972027972027972


In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score


model = svm.SVC()
model.fit(xTrain, yTrain)



yPred = model.predict(xTest)

print("The accuracy of the SVM model:", accuracy_score(yTest, yPred))

The accuracy of the SVM model: 0.5524475524475524


In [43]:
def knn_class(train_df, k, target_col, observation ):
    train_df['distance'] = train_df.apply( lambda x : np.linalg.norm( x.drop(index=target_col) - observation), axis=1 )
    train_df = train_df.sort_values(by='distance').reset_index(drop=True)
    df_k = train_df[0:k]
    return df_k[target_col].value_counts().index[0] 

target_col = 'Survived'
poss_k = [80, 90, 100, 110, 120]
acc_k = np.zeros(len(poss_k))
acc_indx = 0

for k in poss_k:

    predictions = test_df.apply( lambda x : knn_class(train_df.copy(), k, target_col, x.drop(index=target_col)), axis=1)
    
    correct = np.sum(predictions == test_df[target_col])
    acc_k[acc_indx] = correct/len(predictions)
    
    print("k = ", k , "Acc=", acc_k[acc_indx])
    acc_indx = acc_indx + 1
    

k =  80 Acc= 0.6293706293706294
k =  90 Acc= 0.6153846153846154
k =  100 Acc= 0.6153846153846154
k =  110 Acc= 0.6153846153846154
k =  120 Acc= 0.6153846153846154


In [41]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_clf.fit(xTrain, yTrain)
predictions = tree_clf.predict(xTest)
print("The accuracy of the Decision Tree Classifier:", accuracy_score(yTest, predictions))

The accuracy of the Decision Tree Classifier: 0.6993006993006993


In [38]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(n_estimators= 50, learning_rate=1)

model = abc.fit(xTrain, yTrain)

yPred = model.predict(xTest)

score = accuracy_score(yTest, yPred)

print("The accuracy of the model is {}.".format((score)))

The accuracy of the model is 0.6573426573426573.


The most accurate classifier model out of Logistic Regression, Support Vector Machine, kNN, Decision Tree, and the 
AdaBoost Classifier is the Logistic Regression model with an accuracy score of 0.6993006993006993. The next most 
efficient is the AdaBoost Classifier at 0.6573426573426573. Third is the Decision Tree Classifier with an accuracy
score of 0.6993006993006993. Fourth is the Support Vector Machine model or SVM at 0.5874125874125874. Lastly, after
testing a number of different k values, the highest accuracy score was 0.5734265734265734 at k = 55. However, all of
these accuracy scores are quite low, so something should be to done to improve the scores. One way to improve the 
results is to include more features in the data set, more specifically the gender column which had a large impact on 
the survival rate of those aboard the Titanic. 

</br>

Adding the gender feature to the data set greatly improved the accuracy scores of every one of the models and changed
which model is the most accurate. The Decision Tree Classifier is the most efficient after adding the new column with 
an accuracy score of 0.8041958041958042. Second is the Logistic Regression model with 0.7902097902097902. Third is the
AdaBoost Classifier with a score of 0.7822040734018955. Fourth is the kNN model at the k value of 90 with a score of
0.6153846153846154. Last is the SVM model with a score of 0.6013986013986014.